In [ ]:
# Apply the prepocessing
batch_size = 32
train_batches = raw_train.map(format_image).shuffle(1000).batch(batch_size).prefetch(1)
validation_batches = raw_validation.map(format_image).batch(batch_size).prefetch(1)
test_batches = raw_test.map(format_image).batch(batch_size).prefetch(1)

In [ ]:

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(metadata.features['label'].num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
epochs = 10
history = model.fit(
    train_batches,
    epochs=epochs,
    validation_data=validation_batches
)